# Análise de reclamações
Buscamos compreender quais são as empresas mais odiadas.

Autor: Gabriel Bhering

#### Bibliotecas Utilizadas

In [1]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path

Os dados são provenientes do [kaggle](https://www.kaggle.com/gerosa/procon), e um [script](https://gist.github.com/bhering/d21097f5b125d65fd8c422d81695d7f7) foi rodado para higieniza-los. O que o script faz é remover algumas incoerencias (como traços no campo de CEP), e apagar campos sem dados que estavam preenchidos com `NULL` ou `Nao se aplica`. 

Todos os .csv da pasta são lidos e compilados em um único dataframe `df`...

In [2]:
all_files = glob(path.join('./datasets/costumer_complaints',"*.csv"))

In [3]:
df = pd.concat((pd.read_csv(f) for f in all_files))

In [4]:
df[['AnoCalendario', 'DataAbertura', 'strNomeFantasia', 'Atendida', 'DescricaoAssunto', 'DescricaoProblema']][:10]

,AnoCalendario,DataAbertura,strNomeFantasia,Atendida,DescricaoAssunto,DescricaoProblema
0,2012,2009-09-17 10:02:24.000,SAMSUNG,N,"Telefone ( Convencional, Celular, Interfone, E...","Garantia (Abrangência, cobertura, etc.)"
1,2012,2009-09-21 09:01:49.000,AMAZON PC,S,Microcomputador / Produtos de Informática,"Garantia (Abrangência, cobertura, etc.)"
2,2012,2009-09-22 09:49:56.000,AON AFFINITY DO BRASIL,N,Seguro,Não pagamento de indenização (seguros em geral)
3,2012,2009-10-23 15:58:49.000,BOM PRECO,N,Microcomputador / Produtos de Informática,"Garantia (Abrangência, cobertura, etc.)"
4,2012,2009-05-15 12:33:24.000,SUDAMERIS,N,Financeira,Cobrança indevida.
5,2012,2010-02-11 16:35:27.000,NaN,S,Empresa de Cobrança,Cobrança indevida.
6,2012,2010-03-08 12:25:24.000,NaN,S,"Telefone ( Convencional, Celular, Interfone, E...",Falta de peca de reposição
7,2012,2005-08-11 10:16:23.000,SIEMENS CELULARES,N,"Telefone ( Convencional, Celular, Interfone, E...",Falta de peca de reposição
8,2012,2006-11-07 09:07:41.000,LG,S,Televisão / Vídeo Cassete / Filmadora / Video-...,"Garantia (Abrangência, cobertura, etc.)"
9,2012,2010-02-18 16:27:20.000,DAKO / GE,S,Fogão e Microondas,Produto com vício


### Como descrever o quão detestada uma empresa é?
Poderiamos simplesmente analisar a quantidade de reclamações por empresa, embora já de cara isto talvez não seja a melhor forma.

In [5]:
df['strNomeFantasia'].value_counts(normalize=True)[:10]

OI                0.017773
SKY               0.014137
CLARO             0.013978
OI FIXO           0.011000
SAMSUNG           0.009211
VIVO              0.008413
RICARDO ELETRO    0.008254
MOTOROLA          0.007400
TIM CELULAR       0.006362
CCE               0.005699
Name: strNomeFantasia, dtype: float64